### All necessary imports

In [1]:
from lxml.html import parse
from urllib.request import urlopen
from pandas.io.parsers import TextParser
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup #for web scraping
import csv #for writing csv
import logging #for logging
import os
import zipfile
import boto.s3
import sys
from boto.s3.key import Key
import time
import datetime
import logging

### Storing the cik and acc number in variables

In [2]:
ErrorLog = 'errorLogging.log'
logging.basicConfig(filename=ErrorLog,level=logging.DEBUG, filemode = 'w')

In [3]:
file_obj = open("cikInformation.txt","r" )
cik = file_obj.readline().rstrip()
acc = file_obj.readline().rstrip()
if((cik is None or cik is "") or (acc is None or acc is "")):
    logging.info("Cik or access number was left blank. Assigning a IBM's CIK and access key automatically")
    cik = "51143"
    acc = "0000051143-13-000007"
    accNoDash = acc.replace('-', '')
else:
    accNoDash = acc.replace('-', '')

In [4]:
print(cik)
print(accNoDash)

51143
000005114313000007


In [5]:

#Checking if log works: logging.debug('This message should go to the log file')

### Parsing the url 

In [6]:
try:
    url = 'https://www.sec.gov/Archives/edgar/data/'+cik+'/'+accNoDash+'/'+acc+'-index.htm'
    parsed = parse(urlopen(url))
    doc = parsed.getroot()
except:
    logging.critical("Invalid company's url")
    logging.info("Validate that cik number and acces number are correctly inputted")
    exit()

In [7]:
links = doc.findall('.//a')
url = [lnk.get('href')for lnk in doc.findall('.//a')]

In [8]:
url[10:23]

['/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex11.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex12.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex31d1.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex31d2.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex32d1.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex32d2.htm',
 '/Archives/edgar/data/51143/000005114313000007/0000051143-13-000007.txt',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930.xml',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930.xsd',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930_cal.xml',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930_def.xml',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930_lab.xml',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930_pre.xml']

In [9]:
tables = doc.findall('.//table')

In [10]:
example = tables[0]

In [11]:
rows = example.findall('.//tr')

In [12]:
#def _unpack(row, kind='td'):
#    elts = row.findall('.//%s' % kind)
#    return [val.text_content() for val in elts]

#def parse_options_data(table):
#    rows=table.findall('.//tr')
#    header = _unpack(rows[0], kind ='th')
#    data = [_unpack(r) for r in rows[1:]]
#    return TextParser(data, names = header).get_chunk()

#example_data = parse_options_data(example)
#example_data[:10]

#for r in range(len(example_data)):
#        d= example_data['Description'][r]
#        if d =='10-Q' :
#            Doc.append(example_data['Document'][r])



In [16]:
print(_unpack(rows[0], kind="th"))

['Seq', 'Description', 'Document', 'Type', 'Size']


In [14]:
Doc=[]
for t in range(len(tables)):
    example = tables[t]
    
    rows = example.findall('.//tr')
    
    def _unpack(row, kind='td'):
        elts = row.findall('.//%s' % kind)
        return [val.text_content() for val in elts]

    def parse_options_data(table):
        rows=table.findall('.//tr')
        header = _unpack(rows[0], kind ='th')
        data = [_unpack(r) for r in rows[1:]]
        return TextParser(data, names = header).get_chunk()

    example_data = parse_options_data(example)
    
    
    for r in range(len(example_data)):
        d= example_data['Description'][r]
        if d =='10-Q' :
            Doc.append(example_data['Document'][r])

In [15]:
Doc

['ibm13q3_10q.htm']

https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm

## 10-Q Links

In [17]:
for link in Doc:
    try:
        l = 'https://www.sec.gov/Archives/edgar/data/'+cik+'/'+accNoDash+'/'+link+''
        print(l)
        p = parse(urlopen(l))
        q_link= p.getroot()
    except:
        logging.critical("Invalid 10Q url")
        logging.info("Validate that cik number and acces number are correctly inputted")
        exit()

https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm


In [18]:
tables_2 = q_link.findall('.//table')

In [19]:
len(tables_2)

112

In [20]:
ex = tables_2[8]
row = ex.findall('.//tr')

In [21]:
print(_unpack(row[15], kind="td"))

['\n  \xa0\n  ', '\n  Investment\n  in software\n  ', '\n  \xa0\n  ', '\n  \xa0\n  ', '\n  \xa0(406)\xa0\n  ', '\n  \xa0\n  ', '\n  \xa0\n  ', '\n  \xa0(476)\xa0\n  ']


In [22]:
page = urllib.request.urlopen(l)
soup = BeautifulSoup(page,"lxml")
# Grabing all the tables considering the table tags
table_2 = soup.select('div table')

In [23]:
len(table_2)

112

### Traversing through all tables in 10-Q document and storing the tables in the list -> f_tables[ ]

In [24]:
f_tables=[]
for t in table_2:
    #rows t.findAll('tr')
    for row in t.find_all('tr'):
        for cell in row.findAll('td') :
            flag = 0
            if ('$' in cell.get_text().strip() or '%' in cell.get_text().strip()):
                f_tables.append(t)
                flag=1
                break
        if(flag==1):
            break   

In [25]:
len(f_tables)

101

In [26]:
type(f_tables[0])

bs4.element.Tag

In [27]:
#table

In [28]:
def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df
Usage Example

Let’s do an example w

SyntaxError: invalid syntax (<ipython-input-28-8d76a7776035>, line 49)